In [1]:
import wrds
db = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [2]:
lib = db.list_libraries()
lib

['aha_sample',
 'ahasamp',
 'audit',
 'audit_acct_os',
 'audit_common',
 'audit_corp_legal',
 'audit_europe',
 'auditsmp',
 'auditsmp_all',
 'bank',
 'bank_all',
 'bank_premium_samp',
 'banksamp',
 'block',
 'block_all',
 'boardex',
 'boardex_eur',
 'boardex_na',
 'boardex_row',
 'boardex_trial',
 'boardex_uk',
 'boardsmp',
 'bvd',
 'bvd_amadeus_trial',
 'bvd_bvdbankf_trial',
 'bvd_orbis_large',
 'bvd_orbis_medium',
 'bvd_orbis_small',
 'bvd_orbis_trial',
 'bvdsamp',
 'calcbench_trial',
 'calcbnch',
 'cboe',
 'cboe_all',
 'cboe_sample',
 'cboesamp',
 'ciq',
 'ciq_common',
 'ciq_pplintel',
 'ciq_ratings',
 'ciqsamp',
 'ciqsamp_capstrct',
 'ciqsamp_common',
 'ciqsamp_keydev',
 'ciqsamp_pplintel',
 'ciqsamp_ratings',
 'ciqsamp_transactions',
 'ciqsamp_transcripts',
 'cisdmsmp',
 'columnar',
 'comp',
 'comp_bank',
 'comp_bank_daily',
 'comp_execucomp',
 'comp_global',
 'comp_global_daily',
 'comp_na_annual_all',
 'comp_na_daily_all',
 'comp_na_monthly_all',
 'comp_ratings',
 'comp_segments

In [ ]:
financial_data = db.raw_sql("""
    SELECT code, year_, freq, item8306, item1001, item8131, item8231, 
    item8136, item8301, item2003, item3255, item2999, item2301, 
    item1201, item2507, item2509, item11557, item11559, item7546,
    item4601, item1501, item2201, item3101, item7210, item18191,
    item3495, item7250, item3040
    FROM tr_worldscope.wrds_ws_funda
    where year_ >= '2018'
    and year_ <= '2023'
    and year_ in ('2018', '2019', '2020', '2021', '2022', '2023')
""")

# 8306 - profit margin
# 8131 - accounts receivable days
# 8231 - debt to equity
# 1001 - revenue
# 8136 - inventory turnover
# 8301 - return on equity
# 2003 - cash
# 3255 - total debt
# 2301 - property, plant and equipment
# 1201 - R&D
# 2507 - Brands, Patents - Net
# 2509 - Brands, Patents - Gross
# 11557 - Indicator - Restatement Type
# 11559 - Indicator - Restatement Reason
# 7546 - Auditor's Opinion
# code - company
# 4601 - cash flow
# 1501 - cost of goods sold
# 2201 - total current assets
# 3101 - total current liabilities
# 7210 - market capitalisation
# 18191 - EBIT
# 3495 - retained earnings

# 7250 - net income
# 3040 - accounts payable


financial_data

In [ ]:
rename_dict = {
    "code": "code",
    "year_": "year_",
    "freq": "freq",
    "item8306": "profit_margin",
    "item8131": "accounts_receivable_days",
    "item8231": "debt_to_equity",
    "item8136": "inventory_turnover",
    "item8301": "return_on_equity",
    "item2003": "cash",
    "item3255": "total_debt",
    "item2301": "property_plant_and_equipment",
    "item1201": "R&D",
    "item2507": "brands_patents_net",
    "item2509": "brands_patents_gross",
    "item2999": "total_assets",
    "item1001": "revenue",
    "item11557": "restatement_type",
    "item7546": "auditor_opinion",
    "item11559": "restatement_reason",
    "item1501": "cost_of_goods_sold",
    "item4601": "cash_flow",
    "item2201": "total_ca",
    "item3101": "total_cl",
    "item7210": "market_cap",
    "item18191": "EBIT",
    "item3495": "retained_earnings",
    "item7250": "net_income",
    "item3040": "accounts_payable"
}

financial_data.rename(columns = rename_dict, inplace = True)

In [ ]:
# financial_data['restatement_type'] = financial_data['restatement_type'].fillna(0)
# financial_data['restatement_reason'] = financial_data['restatement_reason'].fillna(0)

# financial_data = financial_data.dropna()

financial_data['working_capital'] = financial_data['total_ca'] - financial_data['total_cl']
financial_data['z_score'] = (1.2 * financial_data['working_capital'] / financial_data['total_assets']) + (1.4 * financial_data['retained_earnings'] / financial_data['total_assets']) + (3.3 * financial_data['EBIT'] / financial_data['total_assets']) + (0.6 * financial_data['market_cap'] / financial_data['total_debt']) +  (financial_data['revenue'] / financial_data['total_assets'])

financial_data

,code,year_,freq,profit_margin,revenue,accounts_receivable_days,debt_to_equity,inventory_turnover,return_on_equity,cash,...,cost_of_goods_sold,total_ca,total_cl,market_cap,EBIT,retained_earnings,net_income,accounts_payable,working_capital,Z-Score
0,484.0,2018,A,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,1644138504.12,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,485.0,2018,A,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,490.0,2018,A,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,490.0,2018,B,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,486.0,2018,A,<NA>,80676229000.0,<NA>,143.7189,<NA>,28.77652,<NA>,...,940000.0,<NA>,<NA>,2005844427.888,<NA>,31921794000.0,271559506.24,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435448,162904.0,2023,A,86.69786,848510000.0,<NA>,9.45798,<NA>,9.47471,6973700000.0,...,0.0,<NA>,<NA>,<NA>,127120000.0,630970000.0,1055625.8,<NA>,<NA>,<NA>
435449,162907.0,2023,A,<NA>,0.0,<NA>,36.26886,<NA>,<NA>,1209000.0,...,0.0,14956000.0,857404000.0,1562506.3,<NA>,1349162000.0,-870629.63,57734000.0,-842448000.0,<NA>
435450,162905.0,2023,A,<NA>,0.0,<NA>,0.0,<NA>,<NA>,7876.35,...,0.0,25347292.55,933186.33,619577.58429,-1496638.75,-69502753.19,-18989.26174,284086.33,24414106.22,inf
435451,162906.0,2023,A,56.40848,11555000.0,<NA>,0.07739,<NA>,<NA>,498000.0,...,0.0,<NA>,<NA>,1153163.05605,<NA>,-36852000.0,-40611.29,<NA>,<NA>,<NA>


In [ ]:
company_data = db.raw_sql("""
    SELECT code, item6001, item6011
    FROM tr_worldscope.wrds_ws_company
""")

company_data

,code,item6001,item6011
0,1.0,<NA>,<NA>
1,2.0,<NA>,<NA>
2,3.0,<NA>,<NA>
3,4.0,<NA>,<NA>
4,5.0,<NA>,<NA>
...,...,...,...
131190,162904.0,DAM CAPITAL ADVISORS LTD,4394.0
131191,162905.0,S G N TELECOMS LTD,3750.0
131192,162906.0,UNISHIRE URBAN INFRA LTD,4394.0
131193,162907.0,SHREE HANUMAN SUGAR & INDUSTRIES LTD,8592.0


In [ ]:
rename_dict = {
    "code" : "code",
    "item6011" : "industry",
    "item6001" : "company_name"
}

company_data.rename(columns = rename_dict, inplace = True)

company_data

,code,company_name,industry
0,1.0,<NA>,<NA>
1,2.0,<NA>,<NA>
2,3.0,<NA>,<NA>
3,4.0,<NA>,<NA>
4,5.0,<NA>,<NA>
...,...,...,...
131190,162904.0,DAM CAPITAL ADVISORS LTD,4394.0
131191,162905.0,S G N TELECOMS LTD,3750.0
131192,162906.0,UNISHIRE URBAN INFRA LTD,4394.0
131193,162907.0,SHREE HANUMAN SUGAR & INDUSTRIES LTD,8592.0


In [ ]:
import pandas as pd
import numpy as np

financial_company_data = pd.merge(company_data, financial_data, on='code', how='inner')

relevant_industries = [3400, 3410, 3420, 3430, 3440, 7040, 8550]

# 3400 - drugs, cosmetics & healthcare
# 3410 - diversified
# 3420 - cosmetics & toiletries
# 3430 - ethical drug manufacturers
# 3440 - medical, surgical & dental suppliers
# 7040 - drug store chains
# 8550 - medical services

financial_company_data['industry'] = financial_company_data['industry'].fillna(0).astype(int)

financial_company_data = financial_company_data[financial_company_data['industry'].isin(relevant_industries)]

# financial_company_data = financial_company_data.dropna()
financial_company_data = financial_company_data.dropna(how='all')

financial_company_data[['restatement_type', 'restatement_reason']] = financial_company_data[['restatement_type', 'restatement_reason']].apply(pd.to_numeric, errors='coerce')
numerical_columns = financial_company_data.drop(['code', 'company_name', 'freq'], axis=1).select_dtypes(include=[np.number]).columns
financial_company_data[numerical_columns] = financial_company_data[numerical_columns].fillna(0).replace([np.inf, -np.inf], 0)

financial_company_data

,code,company_name,industry,year_,freq,profit_margin,revenue,accounts_receivable_days,debt_to_equity,inventory_turnover,...,cost_of_goods_sold,total_ca,total_cl,market_cap,EBIT,retained_earnings,net_income,accounts_payable,working_capital,Z-Score
928,677.0,KAZIA THERAPEUTICS LTD,3430,2018,A,-1198.1732,119170.0,10418.21855,2.47099,0.0,...,0.0,9259615.0,3887501.0,15228734.8814,-8544429.0,-12435208.0,-4470186.53598,1406887.0,5372114.0,18.306701
929,677.0,KAZIA THERAPEUTICS LTD,3430,2019,A,-988.59756,99619.0,7778.91983,3.37927,0.0,...,0.0,7514175.0,1900292.0,15279791.13999,-11999542.0,-22459085.0,-7212292.894,1049944.0,5613883.0,16.726451
930,677.0,KAZIA THERAPEUTICS LTD,3430,2020,A,-1545.31371,65905.0,8481.74323,3.39639,0.0,...,0.0,10653601.0,5067473.0,31336882.75103,0.0,-34664446.0,-8604172.31058,1693632.0,5586128.0,0.0
931,677.0,KAZIA THERAPEUTICS LTD,3430,2021,A,91.68924,15224951.0,17.22055,1.24108,0.0,...,0.0,29390818.0,8326554.0,129734853.18132,0.0,-42450023.0,-6318070.1724,1893150.0,21064264.0,0.0
932,677.0,KAZIA THERAPEUTICS LTD,3430,2022,A,-93163.65807,2094.0,15281.28104,0.0,0.0,...,0.0,7608240.0,4685156.0,58891937.29791,-25015895.0,-64989924.0,-17010196.56595,1524174.0,2923084.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435406,162877.0,SEPTERNA INC,8550,2023,A,-461.5894,151000.0,27617.93046,0.0,0.0,...,0.0,112678000.0,6914000.0,0.0,4871000.0,-46576000.0,4180000.0,2637000.0,105764000.0,0.0
435408,162879.0,COSMOS CHOUZAI PHARMACY CO LTD,8550,2023,A,44.07305,5804000000.0,0.0,250.10707,0.0,...,90000000.0,2038000000.0,1300000000.0,0.0,150000000.0,437000000.0,135540.0,678000000.0,738000000.0,0.0
435413,162882.0,SPIRIT SCIENTIFIC CO LTD,3410,2021,A,72.90932,63819000.0,0.0,43.2999,0.0,...,0.0,88945000.0,36619000.0,0.0,-9149000.0,-8660000.0,-368304.64,1673000.0,52326000.0,0.0
435414,162882.0,SPIRIT SCIENTIFIC CO LTD,3410,2022,A,79.96887,123338000.0,33.21128,30.63965,1.72185,...,0.0,121238000.0,46611000.0,0.0,7199000.0,-2627000.0,196736.13,1418000.0,74627000.0,0.0


In [ ]:
financial_company_data.to_csv('FFA_data.csv', index = False)